# Ramen style Clover training with input augmentation

In [1]:
#| default_exp 29-ramen-style-clover-training-with-input-augmentation-for-wikititles

In [2]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
#| export
import os,torch, torch.multiprocessing as mp
from xcai.basics import *
from xcai.models.MMM00X import DBT014, DBT017
from xcai.transform import AugmentMetaInputIdsTfm

In [5]:
os.environ['WANDB_MODE'] = 'disabled'

In [6]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '6,7'
os.environ['WANDB_PROJECT']='xc-nlg_29-ramen-style-clover-training-with-input-augmentation-for-wikititles'

In [7]:
#| export
block = XCBlock.from_cfg('/home/aiscuser/scratch/datasets', 'data_meta', dset='wikititles', valid_pct=0.001, 
                         tfm='xcnlg', tokenizer='distilbert-base-uncased', 
                         smp_features=[('lbl2data',1,2), ('hlk2data',1,1), ('hlk2lbl2data',2,1)], 
                         n_data_meta_samples=50, n_lbl_meta_samples=5)

/home/aiscuser/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")
/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [7]:
block = XCBlock.from_cfg('/home/aiscuser/scratch/datasets', 'data_meta', valid_pct=0.001, 
                         tfm='rm', tokenizer='distilbert-base-uncased', 
                         smp_features=[('lbl2data|hlk2lbl2data',1, 2), ('hlk2data',1, 1)], 
                         n_data_meta_samples=50, n_lbl_meta_samples=5)

/opt/conda/envs/ptca/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/aiscuser/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")
/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
#| export
block = AugmentMetaInputIdsTfm.apply(block, 'hlk_meta', 32, True)

In [33]:
#| export
block.train.dset.data.data_info['input_ids'] = block.train.dset.data.data_info['input_ids_aug_hlk']
block.train.dset.data.data_info['attention_mask'] = block.train.dset.data.data_info['attention_mask_aug_hlk']

block.test.dset.data.data_info['input_ids'] = block.test.dset.data.data_info['input_ids_aug_hlk']
block.test.dset.data.data_info['attention_mask'] = block.test.dset.data.data_info['attention_mask_aug_hlk']

In [17]:
#| export
args = XCLearningArguments(
    output_dir='/home/aiscuser/outputs/29-ramen-style-clover-training-with-input-augmentation-for-wikititles-2-0',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    save_strategy="steps",
    evaluation_strategy='steps',
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.1,
    learning_rate=2e-4,
    generation_num_beams=10,
    generation_length_penalty=1.5,
    predict_with_generation=True,
    representation_search_type='BRUTEFORCE',
    group_by_cluster=True,
    num_clustering_warmup_epochs=1,
    num_cluster_update_epochs=1,
    num_cluster_size_update_epochs=1,
    clustering_type='EXPO',
    minimum_cluster_size=1,
    maximum_cluster_size=300,
    output_concatenation_weight=1.0,
    metric_for_best_model='P@1',
    load_best_model_at_end=True,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    fp16=True,
    label_names=['hlk2data_input_ids', 'hlk2data_attention_mask', 'hlk2data_data2ptr', 'hlk2data_idx', 
                 'phlk2data_idx', 'phlk2data_data2ptr', 
                 'hlk2lbl2data_idx', 'phlk2lbl2data_idx', 'phlk2lbl2data_data2ptr', 
                 'hlk2lbl2data_input_ids', 'hlk2lbl2data_attention_mask', 'hlk2lbl2data_data2ptr'],
)

/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
args = XCLearningArguments(
    output_dir='/home/aiscuser/outputs/29-ramen-style-clover-training-with-input-augmentation-for-wikititles-2-0',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    save_strategy="steps",
    evaluation_strategy='steps',
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.1,
    learning_rate=2e-4,
    generation_num_beams=10,
    generation_length_penalty=1.5,
    predict_with_generation=True,
    representation_search_type='BRUTEFORCE',
    group_by_cluster=True,
    num_clustering_warmup_epochs=1,
    num_cluster_update_epochs=1,
    num_cluster_size_update_epochs=1,
    clustering_type='EXPO',
    minimum_cluster_size=1,
    maximum_cluster_size=300,
    output_concatenation_weight=1.0,
    metric_for_best_model='P@1',
    load_best_model_at_end=True,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    fp16=True,
    label_names=['phlk2lbl_idx', 'phlk2lbl_lbl2ptr', 'hlk2lbl_idx', 'hlk2lbl_input_ids', 
                 'hlk2lbl_attention_mask', 'hlk2lbl_lbl2ptr',
                 'phlk2data_idx', 'phlk2data_data2ptr', 'hlk2data_idx', 'hlk2data_input_ids', 
                 'hlk2data_attention_mask', 'hlk2data_data2ptr'],
)

In [18]:
#| export
test_dset = block.test.dset.sample(n=2000, seed=50)
metric = PrecRecl(block.n_lbl, test_dset.data.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [19]:
#| export
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT014.from_pretrained('distilbert-base-uncased', ig_tok=0, bsz=bsz, tn_targ=10_000, margin=0.3, tau=0.1, 
                               n_negatives=10, apply_softmax=True, lw=100.0, m_lw=0.3, meta_prefix='hlk', 
                               init_drh=True)

Some weights of DBT014 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight', 'gen_lfn.o', 'rep_lfn.u', 'rep_lfn.v']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT017.from_pretrained('distilbert-base-uncased', ig_tok=0, bsz=bsz, tn_targ=10_000, margin=0.3, tau=0.1, 
                               n_negatives=5, apply_softmax=True, lw=100.0, m_lw=0.3, meta_prefix='hlk', 
                               init_drh=True)

In [11]:
#| export
trie = XCTrie.from_block(block)

  0%|          | 0/312330 [00:00<?, ?it/s]

  0%|          | 0/1174296 [00:00<?, ?it/s]

  0%|          | 0/1174296 [00:00<?, ?it/s]

In [12]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    trie=trie,
    train_dataset=block.train.dset,
    eval_dataset=test_dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
learn.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/577 [00:00<?, ?it/s]

node-0:2617957:2617957 [0] NCCL INFO Bootstrap : Using eth0:10.13.32.225<0>
node-0:2617957:2617957 [0] NCCL INFO NET/Plugin : Plugin load (librccl-net.so) returned 2 : librccl-net.so: cannot open shared object file: No such file or directory
node-0:2617957:2617957 [0] NCCL INFO NET/Plugin : No plugin found, using internal implementation
node-0:2617957:2617957 [0] NCCL INFO Kernel version: 5.15.0-1042-azure
RCCL version 2.17.1+hip5.7 HEAD:cbbb3d8+

node-0:2617957:2627797 [1] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

node-0:2617957:2627797 [1] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/transport/net_ib.cc:199 NCCL WARN NET/IB : Unable to open device mlx5_0

node-0:2617957:2627797 [1] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

nod

node-0:2617957:2627796 [0] NCCL INFO Ring 4 : 1 -> 0 -> 1 comm 0xadbce90 nRanks 02 busId d00000
node-0:2617957:2627796 [0] NCCL INFO Ring 5 : 1 -> 0 -> 1 comm 0xadbce90 nRanks 02 busId d00000
node-0:2617957:2627796 [0] NCCL INFO Ring 6 : 1 -> 0 -> 1 comm 0xadbce90 nRanks 02 busId d00000
node-0:2617957:2627796 [0] NCCL INFO Ring 7 : 1 -> 0 -> 1 comm 0xadbce90 nRanks 02 busId d00000
node-0:2617957:2627796 [0] NCCL INFO Ring 8 : 1 -> 0 -> 1 comm 0xadbce90 nRanks 02 busId d00000
node-0:2617957:2627796 [0] NCCL INFO Ring 9 : 1 -> 0 -> 1 comm 0xadbce90 nRanks 02 busId d00000
node-0:2617957:2627796 [0] NCCL INFO Ring 10 : 1 -> 0 -> 1 comm 0xadbce90 nRanks 02 busId d00000
node-0:2617957:2627796 [0] NCCL INFO Ring 11 : 1 -> 0 -> 1 comm 0xadbce90 nRanks 02 busId d00000
node-0:2617957:2627796 [0] NCCL INFO Ring 12 : 1 -> 0 -> 1 comm 0xadbce90 nRanks 02 busId d00000
node-0:2617957:2627796 [0] NCCL INFO Ring 13 : 1 -> 0 -> 1 comm 0xadbce90 nRanks 02 busId d00000
node-0:2617957:2627796 [0] NCCL INFO

/home/aiscuser/scratch/Projects/xcai/xcai/losses.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)
/opt/conda/envs/ptca/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/utils/import_utils.py:521: FutureWarning: `is_torch_tpu_available` is deprecated and will be removed in 4.41.0. Please use the `is_torch_xla_available` instead.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point ope

Step,Training Loss,Validation Loss


  0%|          | 0/261 [00:00<?, ?it/s]

RuntimeError: shape '[1200, -1]' is invalid for input of size 34176312

In [ ]:
#| export
if __name__ == '__main__':
    mp.freeze_support()
    learn.train()